# Import Libraries

In [95]:
import pandas as pd
import seaborn as sns
import re
import requests
from bs4 import BeautifulSoup

## Coleta dos Dados

In [115]:
# Páginas de coleta de dados
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36' }

pg = 6 
html_content = []
for i in range( 1, pg+1 ):
    url = 'https://www.ogol.com.br/edition.php?jornada_in={}&id_edicao=173412&fase=175101'.format( i )
    #url = 'https://www.ogol.com.br/edition.php?jornada_in={}&id_edicao=140879&fase=135852'.format( i )
    print( url )
    response = requests.get(url, headers=headers)

    # Verifique se a solicitação foi bem-sucedida
    if response.status_code == 200:
        print("Solicitação bem-sucedida!")
        html_content.append( response.text )
    else:
        print("Falha na solicitação. Status code:", response.status_code)

https://www.ogol.com.br/edition.php?jornada_in=1&id_edicao=173412&fase=175101
Solicitação bem-sucedida!
https://www.ogol.com.br/edition.php?jornada_in=2&id_edicao=173412&fase=175101
Solicitação bem-sucedida!
https://www.ogol.com.br/edition.php?jornada_in=3&id_edicao=173412&fase=175101
Solicitação bem-sucedida!
https://www.ogol.com.br/edition.php?jornada_in=4&id_edicao=173412&fase=175101
Solicitação bem-sucedida!
https://www.ogol.com.br/edition.php?jornada_in=5&id_edicao=173412&fase=175101
Solicitação bem-sucedida!
https://www.ogol.com.br/edition.php?jornada_in=6&id_edicao=173412&fase=175101
Solicitação bem-sucedida!


## Extração dos Dados

In [116]:
df_classificacao = pd.DataFrame()
for i in range( 0, pg ):
    print( 'Extraindo dados da rodada: {} - Ano 2026'.format( i+1 ) )
    soup = BeautifulSoup(html_content[i], 'html.parser')

    # Encontrando a div com a classe 'edition_table'
    edition_table_div = soup.find('div', id='edition_table')

    # Verificando se a div foi encontrada
    if edition_table_div:
        print( "Div encontrada" )
        # Encontrando a tabela dentro da div
        table = edition_table_div.find('table')

        # Convertendo a tabela HTML para DataFrame
        df = pd.read_html(str(table))[0]

        # Renomeando as colunas
        df1 = df.drop( ['Unnamed: 11'], axis=1 ).rename( columns={'Unnamed: 0': 'Ano', 
                                                                  'Unnamed: 1': 'Rodada',
                                                                  'Unnamed: 2': 'Nome'} )
        # Atribuindo o Ano e Rodada
        df1['Ano'] = 2026
        df1['Rodada'] = i+1
        
        # Unindo as classificacoes das rodadas
        df_classificacao = df_classificacao.append( df1 )
    else:
        print("Div 'edition_table' não encontrada")
        break
        
# Salvando os dados
df_classificacao.to_csv( 'df_eliminatorias_2026.csv', index=0 )

print( 'Dados salvos - DONE' )

Extraindo dados da rodada: 1 - Ano 2026
Div encontrada
Extraindo dados da rodada: 2 - Ano 2026
Div encontrada
Extraindo dados da rodada: 3 - Ano 2026
Div encontrada
Extraindo dados da rodada: 4 - Ano 2026
Div encontrada
Extraindo dados da rodada: 5 - Ano 2026
Div encontrada
Extraindo dados da rodada: 6 - Ano 2026
Div encontrada
Dados salvos - DONE


## Analisando os Dados

In [128]:
df2022 = pd.read_csv( 'df_eliminatorias_2022.csv' )
df2026 = pd.read_csv( 'df_eliminatorias_2026.csv' )

In [129]:
df_history = pd.concat( [df2022, df2026] ).reset_index( drop=True )
df_history.to_csv( 'df_eliminatorias_history.csv', index=0 )

In [127]:
df_history

,Ano,Rodada,Nome,P,J,V,E,D,GM,GC,SG
0,2022,1,Brasil,3,1,1,0,0,5,0,5
1,2022,1,Colômbia,3,1,1,0,0,3,0,3
2,2022,1,Uruguai,3,1,1,0,0,2,1,1
3,2022,1,Argentina,3,1,1,0,0,1,0,1
4,2022,1,Paraguai,1,1,0,1,0,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...
235,2026,6,Brasil,7,6,2,1,3,8,7,1
236,2026,6,Paraguai,5,6,1,2,3,1,3,-2
237,2026,6,Chile,5,6,1,2,3,3,7,-4
238,2026,6,Bolívia,3,6,1,0,5,4,14,-10
